# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#import twint

I decided to use Twint to scrape data from Twitter. I hesitated with Tweepy but this library is limited with the API limitations... As I want to collect all Twitter mentions from September, that library might not be well-suited.

# Twitter data collection

In [4]:
#configuration
#config = twint.Config()
#config.Search = "PS5"
#config.Lang = "en"
#config.Limit = 5
#config.Since = '2020–09–01 00:01:00'
#config.Until = "2020–9–28 00:00:00"
#config.Store_csv = True
#config.Output = "test.csv"
#running search
#twint.run.Search(config)

In [5]:
#import twitter_scraper

In [6]:
#from twitter_scraper import get_tweets

In [7]:
#tweets = get_tweets("PS5",pages=5)

In [8]:
#tweets

In [9]:
#tweets_df = pd.DataFrame()

In [10]:
#for tweet in tweets:
    #print('Keys:', list(tweet.keys()), '\n')
    #break

In [11]:
#for tweet in tweets:
    #_ = pd.DataFrame({'text' : [tweet['text']],
                    #'isRetweet' : tweet['isRetweet'],
                    #'replies' : tweet['replies'],
                    #'retweets' : tweet['retweets'],
                    #'likes' : tweet['likes']
                    #})
    #tweets_df = tweets_df.append(_, ignore_index = True)
#tweets_df.head()

In [12]:
#for i in tweets:
    #tweets_df.append(i)

In [13]:
#tweets_df

In [14]:
#import twitterscraper
#import datetime as dt

In [15]:
#from twitterscraper import query_tweets

In [16]:
#tweets = query_tweets('bitcoin', limit=10, begindate=dt.date.today(), enddate=dt.date.today(), lang='en')

In [17]:
#for i in tweets:
    #print(i)

In [18]:
#df = pd.DataFrame(t.__dict__ for t in tweets)

In [19]:
#df

In [20]:
#import GetOldTweets3 as got

In [21]:
#tweetCriteria = got.manager.TweetCriteria().setQuerySearch('PS5')\
                                           #.setSince("2020-09-01")\
                                           #.setUntil("2020-09-30")\
                                           #.setMaxTweets(10)
#tweet = got.manager.TweetManager.getTweets(tweetCriteria)[0]
#print(tweet.text)

In [22]:
#import twint
#c = twint.Config()
#c.Search = "ps5"
#c.Store_object = True
#c.Limit = 100
#twint.run.Search(c)
#tlist = c.search_tweet_list

In [23]:
#c=twint.Config()
#c.Hide_output=False #but it ignores this
#c.Store_object=True
#c.Store_csv=True
#c.Since="2020-09-01"
#c.User_full=True
#c.Limit=100 #I have tested up to 1000 ok
#c.Output="testing.csv" #but it ignores this

#srchname="(from:RealDonaldTrump)"
#srch=srchname+" max_id:"+str(1307047063197224961)

#twint.run.Search(c)
#tlist = c.search_tweet_list
#print(tlist)

# YouTube data extraction

At first, I wanted to analyze all PS5 and Xbox tweets in September as many announcements occured that month (price, games, etc.). But it was too difficult for me to get the Twitter data. The API limitations only let me have access to the last 7 days of data. I also had a look at the scrapers (Twint, tweetscraper, GetOldTweets3, etc.) but they are all down for some reasons...
Therefore, I decided to change a bit the scope of the project.

I will have a look at the YouTube comments of both PS5 and Xbox games showcases.
The PS5 showcase occured on the 16th of September while the Xbox Series X showcase occured on the 23rd of July.

In [24]:
Xbox_data = pd.read_fwf('Xbox_comments.txt',header=None)

In [25]:
Xbox_data.fillna('',inplace=True)

In [26]:
#Consolidating text together in one column
Xbox_data['Text'] = Xbox_data[2]
Xbox_data.drop(2,axis=1,inplace=True)
for i in range(3,21):
    Xbox_data['Text']=Xbox_data['Text'] + " " + Xbox_data[i]
    Xbox_data.drop(i,axis=1,inplace=True)

In [27]:
#Removing ":" from end of column "1" (hours)
Xbox_data[1]=Xbox_data[1].apply(lambda x: x[:-1])

In [28]:
#Renaming columns
Xbox_data.rename(columns={0:'Date',1:'Hour'},inplace=True)

In [29]:
Xbox_data.tail(60)

,Date,Hour,Text
1093,2020-08-26,16:01:42,AhahahahahahahahaHhahahaHHHhHHahahahahahahahah...
1094,2020-08-27,21:48:40,#FALLGUYSONXBOX
1095,2020-08-30,09:46:18,RIP
1096,2020-08-30,14:29:26,6:34
1097,2020-08-30,16:26:11,xbox took an L
1098,2020-08-30,23:57:07,Hola
1099,2020-09-01,16:18:27,Thank you for using Persian language ...
1100,2020-09-01,16:18:47,Thank you for using Persian ️️️
1101,2020-09-05,15:13:16,Lol ripbox serious x with failo infuckety ...
1102,2020-09-08,18:32:33,6:25 sounds just like the Cabal in Destiny! ...


# Text processing

In [30]:
#Creating a word cloud
#from wordcloud import WordCloud

#full_text = ' '.join([txt for txt in Xbox_data['Text']])
#wordcloud = WordCloud(width=500, height=300, max_font_size=119).generate(full_text)

#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis('off')
#plt.show()

In [31]:
import nltk
import re

In [32]:
#We need to install the "stopwords" corpus in order to remove the stopwords later on
#nltk.download()

In [33]:
#Creating a function to clean the comments
def clean_text(text):
    text = re.sub(r'https?:\/\/\S+', '', text) #Removing hyperlinks
    text = re.sub(r"\d{2}:\d{2}s*", '', text) #Removing timestamps XX:XX
    text = re.sub(r"\d{1}:\d{2}s*", '', text) #Removing timestamps X:XX
    text = re.sub('[^A-Za-z0-9 ]+', '', text) #Removing special characters
    return text

In [34]:
Xbox_data['Text']=Xbox_data['Text'].apply(clean_text)

In [35]:
Xbox_data.head(60)

,Date,Hour,Text
0,2020-07-23,16:59:51,First
1,2020-07-23,16:59:55,bad
2,2020-07-23,16:59:56,First
3,2020-07-23,16:59:58,First comment
4,2020-07-23,17:00:02,Hello
5,2020-07-23,17:00:03,28 seconds oh man
6,2020-07-23,17:00:05,Tetris time
7,2020-07-23,17:00:06,Seahawks shouldve ran the ball
8,2020-07-23,17:00:08,More trailers Yay
9,2020-07-23,17:00:10,HYPE


## Tokenizer

In [36]:
Xbox_data_tokenized = Xbox_data['Text'].apply(nltk.word_tokenize)

In [44]:
Xbox_data_tokenized.head(60)

0                                               [First]
1                                                 [bad]
2                                               [First]
3                                      [First, comment]
4                                               [Hello]
5                                [28, seconds, oh, man]
6                                        [Tetris, time]
7                  [Seahawks, shouldve, ran, the, ball]
8                                 [More, trailers, Yay]
9                                                [HYPE]
10                       [One, of, the, first, comment]
11                                          [EXCELLENT]
12                               [Halo, looks, amazing]
13                                [Xbox, Takes, The, W]
14                                    [Lets, go, fable]
15                                       [This, stinks]
16    [Fps, fps, fps, fps, fps, fps, fps, fps, fps, ...
17                                              

## Part-of-Speech tagging (POS)

I need to perform POS to help the stemmer/lemmatizer to detect verbs and keywords...

In [39]:
from nltk import pos_tag

In [48]:
Xbox_data_pos = Xbox_data_tokenized.apply(pos_tag)

In [50]:
Xbox_data_pos.head(60)

0                                         [(First, RB)]
1                                           [(bad, JJ)]
2                                         [(First, RB)]
3                          [(First, JJ), (comment, NN)]
4                                         [(Hello, NN)]
5       [(28, CD), (seconds, NNS), (oh, JJ), (man, NN)]
6                           [(Tetris, NNP), (time, NN)]
7     [(Seahawks, NNP), (shouldve, NN), (ran, VBD), ...
8            [(More, JJR), (trailers, NNS), (Yay, VBP)]
9                                          [(HYPE, NN)]
10    [(One, CD), (of, IN), (the, DT), (first, JJ), ...
11                                   [(EXCELLENT, NNS)]
12           [(Halo, NNP), (looks, VBZ), (amazing, JJ)]
13      [(Xbox, NN), (Takes, VBZ), (The, DT), (W, NNP)]
14                [(Lets, NNS), (go, VBP), (fable, JJ)]
15                          [(This, DT), (stinks, NNS)]
16    [(Fps, NNP), (fps, NN), (fps, NN), (fps, NN), ...
17                                       [(Gears

## Lemmatizer

In [76]:
from nltk.stem import WordNetLemmatizer 

In [97]:
Xbox_data_tokenized.shape[0]

1153

In [92]:
lm = WordNetLemmatizer()
#Xbox_data_lemmatized = Xbox_data_pos.astype(str).apply(lm.lemmatize())
#lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
Xbox_data_lemmatized = Xbox_data_tokenized.apply(' '.join([lm.lemmatize(w) for w in Xbox_data_tokenized.astype(str)]))


for i in (0,Xbox_data_tokenized.shape[0]+1):
    


AttributeError: '['First'] ['bad'] ['First'] ['First', 'comment'] ['Hello'] ['28', 'seconds', 'oh', 'man'] ['Tetris', 'time'] ['Seahawks', 'shouldve', 'ran', 'the', 'ball'] ['More', 'trailers', 'Yay'] ['HYPE'] ['One', 'of', 'the', 'first', 'comment'] ['EXCELLENT'] ['Halo', 'looks', 'amazing'] ['Xbox', 'Takes', 'The', 'W'] ['Lets', 'go', 'fable'] ['This', 'stinks'] ['Fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps'] ['Gears'] ['That', 'was', 'unimpressive'] ['Why', 'are', 'there', '40', 'likes', 'but', '11', 'views'] ['FABLE', 'Fable', 'what', 'Fable', '4', 'Fable', 'remake'] [] ['Forza', 'and', 'Fable'] ['Just', 'amazing'] ['Well', 'that', 'was', 'really', 'disappointing', 'The', 'ps5', 'event', 'wins', 'for', 'sure'] ['Like', 'For', 'Xbox', 'Comment', 'For', 'Ps5'] ['take', 'my', 'money'] ['22nd'] ['Once', 'more', 'Xbox', 'annihilates', 'the', 'piss', 'station', 'again', 'and', 'again'] ['Xbox', 'takes', 'a', 'L'] ['r

In [88]:
Xbox_data_lemmatized

"['First'] ['bad'] ['First'] ['First', 'comment'] ['Hello'] ['28', 'seconds', 'oh', 'man'] ['Tetris', 'time'] ['Seahawks', 'shouldve', 'ran', 'the', 'ball'] ['More', 'trailers', 'Yay'] ['HYPE'] ['One', 'of', 'the', 'first', 'comment'] ['EXCELLENT'] ['Halo', 'looks', 'amazing'] ['Xbox', 'Takes', 'The', 'W'] ['Lets', 'go', 'fable'] ['This', 'stinks'] ['Fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps', 'fps'] ['Gears'] ['That', 'was', 'unimpressive'] ['Why', 'are', 'there', '40', 'likes', 'but', '11', 'views'] ['FABLE', 'Fable', 'what', 'Fable', '4', 'Fable', 'remake'] [] ['Forza', 'and', 'Fable'] ['Just', 'amazing'] ['Well', 'that', 'was', 'really', 'disappointing', 'The', 'ps5', 'event', 'wins', 'for', 'sure'] ['Like', 'For', 'Xbox', 'Comment', 'For', 'Ps5'] ['take', 'my', 'money'] ['22nd'] ['Once', 'more', 'Xbox', 'annihilates', 'the', 'piss', 'station', 'again', 'and', 'again'] ['Xbox', 'takes', 'a', 'L'] ['release', 'xbox',

## Stemming

In [75]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
#Xbox_data_stemmed = [ps.stem(text).lower() for text in str(Xbox_data_tokenized)]
Xbox_data_stemmed = Xbox_data_pos.astype(str).apply(ps.stem(text) for text,test in Xbox_data_pos)

TypeError: 'generator' object is not callable

In [72]:
Xbox_data_stemmed

0                                       [('first', 'rb')]
1                                         [('bad', 'jj')]
2                                       [('first', 'rb')]
3                    [('first', 'jj'), ('comment', 'nn')]
4                                       [('hello', 'nn')]
                              ...                        
1148    [('pyschonauts', 'nnp'), ('looks', 'vbz'), ('m...
1149    [('cross', 'nnp'), ('fire', 'nn'), ('looks', '...
1150    [('whats', 'in'), ('the', 'dt'), ('song', 'nn'...
1151    [('why', 'wrb'), ('are', 'vbp'), ('people', 'n...
1152    [('imagine', 'nn'), ('fallout', 'nnp'), ('and'...
Name: Text, Length: 1153, dtype: object

In [58]:
Xbox_data_stemmed.iloc[39]

"[('im', 'nnp'), ('really', 'rb'), ('interesting', 'vbg'), ('in', 'in'), ('crossfire', 'nn'), ('x', 'nn'), ('but', 'cc'), ('i', 'prp'), ('was', 'vbd'), ('kinda', 'rb'), ('disappointed', 'jj'), ('because', 'in'), ('we', 'prp'), ('didnt', 'vbp'), ('saw', 'jj'), ('cuphead', 'nn'), ('killer', 'nnp'), ('instinct', 'nnp'), ('elden', 'jj'), ('ring', 'nn'), ('banjo', 'nn'), ('conker', 'nn'), ('forza', 'nn'), ('gameplay', 'nn'), ('or', 'cc'), ('hellblade', 'vb'), ('2', 'cd'), ('gameplay', 'nn'), ('but', 'cc'), ('im', 'nnp'), ('really', 'rb'), ('waiting', 'vbg'), ('for', 'in'), ('these', 'dt'), ('game', 'nn'), ('you', 'prp'), ('have', 'vbp'), ('shown', 'vbn'), ('to', 'to'), ('us', 'prp'), ('thnxx', 'vb')]"

In [284]:
for word in Xbox_data_tokenized.iloc[39]:
    print(ps.stem(word))

Im
realli
interest
in
crossfir
x
but
I
wa
kinda
disappoint
becaus
we
didnt
saw
cuphead
killer
instinct
elden
ring
banjo
conker
forza
gameplay
or
hellblad
2
gameplay
but
Im
realli
wait
for
these
game
you
have
shown
to
us
thnxx
